In [96]:
import sys
sys.path.append("../")

import pandas as pd
import os
import statsmodels.formula.api as sm
import statsmodels.formula.api as smf

from library import regulations
from library import characteristics
from library import analysis
from library import tables

from openpyxl import load_workbook
pd.set_option('display.max_columns', None)

In [97]:
data_path = '/Users/kylieleblancKylie/domino/dofis/data/'
table_path = '/Users/kylieleblancKylie/domino/dofis/results/Who Needs Rules/'
data = pd.read_csv(os.path.join(data_path, 'clean', 'master_data_district.csv'),
                  sep=",")
data = data[data.doi == 1]
data.sample(5)

/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,year,district,distname,distischarter,rating_academic,rating_financial,type,type_description,cntyname,students_amind_num,students_asian_num,students_black_num,students_cte_num,students_ell_num,students_frpl_num,students_hisp_num,students_num,students_paci_num,students_sped_num,students_tworaces_num,students_white_num,teachers_badegree_num,teachers_exp_ave,teachers_msdegree_num,teachers_new_num,teachers_nodegree_num,teachers_num,teachers_phddegree_num,teachers_tenure_ave,teachers_turnover_denom,teachers_turnover_num,teachers_turnover_ratio_d,alg_avescore,bio_avescore,eng1_avescore,eng2_avescore,m_3rd_avescore,m_4th_avescore,m_5th_avescore,m_6th_avescore,m_7th_avescore,m_8th_avescore,r_3rd_avescore,r_4th_avescore,r_5th_avescore,r_6th_avescore,r_7th_avescore,r_8th_avescore,s_5th_avescore,s_8th_avescore,us_avescore,alg_numtakers,bio_numtakers,eng1_numtakers,eng2_numtakers,m_3rd_numtakers,m_4th_numtakers,m_5th_numtakers,m_6th_numtakers,m_7th_numtakers,m_8th_numtakers,r_3rd_numtakers,r_4th_numtakers,r_5th_numtakers,r_6th_numtakers,r_7th_numtakers,r_8th_numtakers,s_5th_numtakers,s_8th_numtakers,us_numtakers,class_size_1,class_size_2,class_size_3,class_size_4,class_size_5,class_size_6,class_size_k,class_size_sec_lang,class_size_sec_math,class_size_sec_r,class_size_sec_sci,class_size_sec_ss,stu_teach_ratio,Unnamed: 0.1,link,p_doi,doi_date,reg11_1511,reg11_162,reg11_164,reg11_251,reg11_252,reg11_253,reg11_255,reg11_521,reg12_002,reg12_003,reg12_0522,reg12_131,reg15_113,reg19_231,reg20_085,reg21_0,reg21_0003,reg21_002,reg21_003,reg21_0031,reg21_004,reg21_005,reg21_0051,reg21_0203,reg21_033,reg21_04,reg21_0401,reg21_044,reg21_0442,reg21_048,reg21_049,reg21_0491,reg21_051,reg21_052,reg21_053,reg21_055,reg21_056,reg21_057,reg21_0583,reg21_081,reg21_0811,reg21_101,reg21_102,reg21_106,reg21_111,reg21_158,reg21_202,reg21_203,reg21_211,reg21_252,reg21_342,reg21_351,reg21_352,reg21_3521,reg21_353,reg21_354,reg21_3541,reg21_355,reg21_40,reg21_401,reg21_4016,reg21_402,reg21_4021,reg21_404,reg21_405,reg21_407,reg21_415,reg21_451,reg21_4513,reg21_458,reg21401,reg22_004,reg22_007,reg22_011,reg22_021,reg22_0831,reg22_0833,reg22_102,reg22_111,reg23_354,reg23_3541,reg24_401,reg25_001,reg25_01812,reg25_035,reg25_036,reg25_038,reg25_043,reg25_08,reg25_081,reg25_0811,reg25_0812,reg25_082,reg25_083,reg25_084,reg25_085,reg25_086,reg25_087,reg25_0915,reg25_092,reg25_094,reg25_111,reg25_1111,reg25_1113,reg25_112,reg25_113,reg25_114,reg25_206,reg25_250,reg25_354,reg25_451,reg25_801,reg25_8011,reg25_8012,reg25_811,reg25_812,reg25_902,reg26_011,reg26_0811,reg28_002,reg28_0021,reg28_004,reg28_014,reg28_0214,reg28_0216,reg28_023,reg28_025,reg28_026,reg28_027,reg28_0811,reg28_214,reg28_216,reg29_0821,reg29_0822,reg29_092,reg29_151,reg29_916,reg36_054,reg37_001,reg37_0012,reg37_002,reg37_005,reg37_006,reg37_007,reg37_008,reg37_0081,reg37_0082,reg37_00821,reg37_0087,reg37_01,reg37_010,reg37_027,reg37_105,reg37_107,reg37_152,reg38_004,reg38_0041,reg38_006,reg38_101,reg38_102,reg39_022,reg39_053,reg39_082,reg39_131,reg39_306,reg39_362,reg41_401,reg42_005,reg42_006,reg42_205,reg44_031,reg44_0331,reg44_0352,reg44_042,reg44_043,reg44_047,reg44_901,reg44_902,reg44_903,reg45_204,reg45_205,reg45_206,reg45_207,reg45_208,reg45_209,reg46_001,reg51_403,reg55_0811,reg97_1001,total,doi_year,_merge,doi,county,cnty_pop,students_frpl,students_black,students_hisp,students_white,students_ell,students_sped,students_cte,students_teacher_ratio,geography,charter,district_status,type_urban,type_suburban,type_town,type_rural,eligible,teachers_nodegree,teachers_badegree,teachers_msdegree,teachers_phddegree,r_3rd_std,m_3rd_std,r_4th_std,m_4th_std,r_5th_std,m_5th_std,r_6th_std,m_6th_std,r_7th_std,m_7th_std,r_8th_std,m_8th_std,s_8th_std,alg_std,bio_std,eng1_std,eng2_std,us_std,elem_math,elem_reading,elem,middle_math,middle_reading,middle_science,algebra,biology,eng1,avescores,treatpost,yearpost,yearpre,pre5,pre4,pre3,pre2,pre1,post1,post2,post3
6238,6238,2017,23902,SILVERTON ISD,N,M,Pa

In [98]:
data_pre = data.loc[data.pre1 == 1]
data_pre['pre_score'] = data_pre.groupby('district')['avescores'].transform('mean')
data_pre = data_pre.drop_duplicates(subset = 'district')
data_pre = data_pre.rename(columns = {'students_hisp': 'pre_hisp',
                                     'teachers_turnover_ratio_d': 'pre_turnover',
                                     'students_ell': 'pre_ell',
                                      'students_white': 'pre_white',
                                      'students_black': 'pre_black',
                                     'students_sped': 'pre_sped',
                                      'students_frpl': 'pre_frpl',
                                     'avescores': 'pre_avescore',
                                    'students_num': 'pre_num',
                                     'teachers_tenure_ave': 'pre_tenure',
                                      'students_teacher_ratio': 'pre_sturatio'
                                     })
data_pre.sample()

/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


,Unnamed: 0,year,district,distname,distischarter,rating_academic,rating_financial,type,type_description,cntyname,students_amind_num,students_asian_num,students_black_num,students_cte_num,students_ell_num,students_frpl_num,students_hisp_num,pre_num,students_paci_num,students_sped_num,students_tworaces_num,students_white_num,teachers_badegree_num,teachers_exp_ave,teachers_msdegree_num,teachers_new_num,teachers_nodegree_num,teachers_num,teachers_phddegree_num,pre_tenure,teachers_turnover_denom,teachers_turnover_num,pre_turnover,alg_avescore,bio_avescore,eng1_avescore,eng2_avescore,m_3rd_avescore,m_4th_avescore,m_5th_avescore,m_6th_avescore,m_7th_avescore,m_8th_avescore,r_3rd_avescore,r_4th_avescore,r_5th_avescore,r_6th_avescore,r_7th_avescore,r_8th_avescore,s_5th_avescore,s_8th_avescore,us_avescore,alg_numtakers,bio_numtakers,eng1_numtakers,eng2_numtakers,m_3rd_numtakers,m_4th_numtakers,m_5th_numtakers,m_6th_numtakers,m_7th_numtakers,m_8th_numtakers,r_3rd_numtakers,r_4th_numtakers,r_5th_numtakers,r_6th_numtakers,r_7th_numtakers,r_8th_numtakers,s_5th_numtakers,s_8th_numtakers,us_numtakers,class_size_1,class_size_2,class_size_3,class_size_4,class_size_5,class_size_6,class_size_k,class_size_sec_lang,class_size_sec_math,class_size_sec_r,class_size_sec_sci,class_size_sec_ss,stu_teach_ratio,Unnamed: 0.1,link,p_doi,doi_date,reg11_1511,reg11_162,reg11_164,reg11_251,reg11_252,reg11_253,reg11_255,reg11_521,reg12_002,reg12_003,reg12_0522,reg12_131,reg15_113,reg19_231,reg20_085,reg21_0,reg21_0003,reg21_002,reg21_003,reg21_0031,reg21_004,reg21_005,reg21_0051,reg21_0203,reg21_033,reg21_04,reg21_0401,reg21_044,reg21_0442,reg21_048,reg21_049,reg21_0491,reg21_051,reg21_052,reg21_053,reg21_055,reg21_056,reg21_057,reg21_0583,reg21_081,reg21_0811,reg21_101,reg21_102,reg21_106,reg21_111,reg21_158,reg21_202,reg21_203,reg21_211,reg21_252,reg21_342,reg21_351,reg21_352,reg21_3521,reg21_353,reg21_354,reg21_3541,reg21_355,reg21_40,reg21_401,reg21_4016,reg21_402,reg21_4021,reg21_404,reg21_405,reg21_407,reg21_415,reg21_451,reg21_4513,reg21_458,reg21401,reg22_004,reg22_007,reg22_011,reg22_021,reg22_0831,reg22_0833,reg22_102,reg22_111,reg23_354,reg23_3541,reg24_401,reg25_001,reg25_01812,reg25_035,reg25_036,reg25_038,reg25_043,reg25_08,reg25_081,reg25_0811,reg25_0812,reg25_082,reg25_083,reg25_084,reg25_085,reg25_086,reg25_087,reg25_0915,reg25_092,reg25_094,reg25_111,reg25_1111,reg25_1113,reg25_112,reg25_113,reg25_114,reg25_206,reg25_250,reg25_354,reg25_451,reg25_801,reg25_8011,reg25_8012,reg25_811,reg25_812,reg25_902,reg26_011,reg26_0811,reg28_002,reg28_0021,reg28_004,reg28_014,reg28_0214,reg28_0216,reg28_023,reg28_025,reg28_026,reg28_027,reg28_0811,reg28_214,reg28_216,reg29_0821,reg29_0822,reg29_092,reg29_151,reg29_916,reg36_054,reg37_001,reg37_0012,reg37_002,reg37_005,reg37_006,reg37_007,reg37_008,reg37_0081,reg37_0082,reg37_00821,reg37_0087,reg37_01,reg37_010,reg37_027,reg37_105,reg37_107,reg37_152,reg38_004,reg38_0041,reg38_006,reg38_101,reg38_102,reg39_022,reg39_053,reg39_082,reg39_131,reg39_306,reg39_362,reg41_401,reg42_005,reg42_006,reg42_205,reg44_031,reg44_0331,reg44_0352,reg44_042,reg44_043,reg44_047,reg44_901,reg44_902,reg44_903,reg45_204,reg45_205,reg45_206,reg45_207,reg45_208,reg45_209,reg46_001,reg51_403,reg55_0811,reg97_1001,total,doi_year,_merge,doi,county,cnty_pop,pre_frpl,pre_black,pre_hisp,pre_white,pre_ell,pre_sped,students_cte,pre_sturatio,geography,charter,district_status,type_urban,type_suburban,type_town,type_rural,eligible,teachers_nodegree,teachers_badegree,teachers_msdegree,teachers_phddegree,r_3rd_std,m_3rd_std,r_4th_std,m_4th_std,r_5th_std,m_5th_std,r_6th_std,m_6th_std,r_7th_std,m_7th_std,r_8th_std,m_8th_std,s_8th_std,alg_std,bio_std,eng1_std,eng2_std,us_std,elem_math,elem_reading,elem,middle_math,middle_reading,middle_science,algebra,biology,eng1,pre_avescore,treatpost,yearpost,yearpre,pre5,pre4,pre3,pre2,pre1,post1,post2,post3,pre_score
6290,6290,2017,37908,NEW SUMMERFIELD ISD,N,M,Pass,H,RURAL,CHEROKEE,1,3,14,144,222,476,455,544,0

In [99]:
variables = ['district']
variables = variables + (list(data_pre.filter(regex = ("pre_"))))
data_pre = data_pre[variables]
data_pre = data_pre.merge(data[data.year == 2016][['district', 'type_urban', 'type_suburban', 'type_town', 'type_rural']], 
                          how = 'left', left_on = ['district'], right_on = ['district'])
data_pre = data_pre.rename(columns = {'type_urban': 'pre_urban',
                                      'type_suburban': 'pre_suburban',
                                      'type_town': 'pre_town',
                                      'type_rural': 'pre_rural'})
data_pre['pre_turnover'] = data_pre.pre_turnover / 100
data_pre.sample(5)

,district,pre_num,pre_tenure,pre_turnover,pre_frpl,pre_black,pre_hisp,pre_white,pre_ell,pre_sped,pre_sturatio,pre_avescore,pre_score,pre_urban,pre_suburban,pre_town,pre_rural
74,91902,540,7.5,0.192,0.512963,0.020370,0.185185,0.759259,0.053704,0.053704,12.949640,0.628403,0.628403,0,0,0,1
713,71908,1111,7.0,0.169,0.945995,0.000000,0.995500,0.003600,0.538254,0.063006,13.974843,-0.397310,-0.397310,0,0,1,0
384,103902,232,6.2,0.149,0.616379,0.000000,0.577586,0.418103,0.288793,0.043103,10.995261,0.947774,0.947774,0,0,0,1
766,176903,576,13.0,0.176,0.682292,0.013889,0.038194,0.925347,0.000000,0.149306,11.338583,0.188831,0.188831,0,0,0,1
719,84908,1648,3.5,0.319,0.819782,0.444175,0.305218,0.224515,0.058859,0.116505,15.147059,-0.494940,-0.494940,0,1,0,0


In [100]:
data = data.reset_index().merge(data_pre, left_on = 'district', right_on = 'district', how = 'left')
data.head(5)

,index,Unnamed: 0,year,district,distname,distischarter,rating_academic,rating_financial,type,type_description,cntyname,students_amind_num,students_asian_num,students_black_num,students_cte_num,students_ell_num,students_frpl_num,students_hisp_num,students_num,students_paci_num,students_sped_num,students_tworaces_num,students_white_num,teachers_badegree_num,teachers_exp_ave,teachers_msdegree_num,teachers_new_num,teachers_nodegree_num,teachers_num,teachers_phddegree_num,teachers_tenure_ave,teachers_turnover_denom,teachers_turnover_num,teachers_turnover_ratio_d,alg_avescore,bio_avescore,eng1_avescore,eng2_avescore,m_3rd_avescore,m_4th_avescore,m_5th_avescore,m_6th_avescore,m_7th_avescore,m_8th_avescore,r_3rd_avescore,r_4th_avescore,r_5th_avescore,r_6th_avescore,r_7th_avescore,r_8th_avescore,s_5th_avescore,s_8th_avescore,us_avescore,alg_numtakers,bio_numtakers,eng1_numtakers,eng2_numtakers,m_3rd_numtakers,m_4th_numtakers,m_5th_numtakers,m_6th_numtakers,m_7th_numtakers,m_8th_numtakers,r_3rd_numtakers,r_4th_numtakers,r_5th_numtakers,r_6th_numtakers,r_7th_numtakers,r_8th_numtakers,s_5th_numtakers,s_8th_numtakers,us_numtakers,class_size_1,class_size_2,class_size_3,class_size_4,class_size_5,class_size_6,class_size_k,class_size_sec_lang,class_size_sec_math,class_size_sec_r,class_size_sec_sci,class_size_sec_ss,stu_teach_ratio,Unnamed: 0.1,link,p_doi,doi_date,reg11_1511,reg11_162,reg11_164,reg11_251,reg11_252,reg11_253,reg11_255,reg11_521,reg12_002,reg12_003,reg12_0522,reg12_131,reg15_113,reg19_231,reg20_085,reg21_0,reg21_0003,reg21_002,reg21_003,reg21_0031,reg21_004,reg21_005,reg21_0051,reg21_0203,reg21_033,reg21_04,reg21_0401,reg21_044,reg21_0442,reg21_048,reg21_049,reg21_0491,reg21_051,reg21_052,reg21_053,reg21_055,reg21_056,reg21_057,reg21_0583,reg21_081,reg21_0811,reg21_101,reg21_102,reg21_106,reg21_111,reg21_158,reg21_202,reg21_203,reg21_211,reg21_252,reg21_342,reg21_351,reg21_352,reg21_3521,reg21_353,reg21_354,reg21_3541,reg21_355,reg21_40,reg21_401,reg21_4016,reg21_402,reg21_4021,reg21_404,reg21_405,reg21_407,reg21_415,reg21_451,reg21_4513,reg21_458,reg21401,reg22_004,reg22_007,reg22_011,reg22_021,reg22_0831,reg22_0833,reg22_102,reg22_111,reg23_354,reg23_3541,reg24_401,reg25_001,reg25_01812,reg25_035,reg25_036,reg25_038,reg25_043,reg25_08,reg25_081,reg25_0811,reg25_0812,reg25_082,reg25_083,reg25_084,reg25_085,reg25_086,reg25_087,reg25_0915,reg25_092,reg25_094,reg25_111,reg25_1111,reg25_1113,reg25_112,reg25_113,reg25_114,reg25_206,reg25_250,reg25_354,reg25_451,reg25_801,reg25_8011,reg25_8012,reg25_811,reg25_812,reg25_902,reg26_011,reg26_0811,reg28_002,reg28_0021,reg28_004,reg28_014,reg28_0214,reg28_0216,reg28_023,reg28_025,reg28_026,reg28_027,reg28_0811,reg28_214,reg28_216,reg29_0821,reg29_0822,reg29_092,reg29_151,reg29_916,reg36_054,reg37_001,reg37_0012,reg37_002,reg37_005,reg37_006,reg37_007,reg37_008,reg37_0081,reg37_0082,reg37_00821,reg37_0087,reg37_01,reg37_010,reg37_027,reg37_105,reg37_107,reg37_152,reg38_004,reg38_0041,reg38_006,reg38_101,reg38_102,reg39_022,reg39_053,reg39_082,reg39_131,reg39_306,reg39_362,reg41_401,reg42_005,reg42_006,reg42_205,reg44_031,reg44_0331,reg44_0352,reg44_042,reg44_043,reg44_047,reg44_901,reg44_902,reg44_903,reg45_204,reg45_205,reg45_206,reg45_207,reg45_208,reg45_209,reg46_001,reg51_403,reg55_0811,reg97_1001,total,doi_year,_merge,doi,county,cnty_pop,students_frpl,students_black,students_hisp,students_white,students_ell,students_sped,students_cte,students_teacher_ratio,geography,charter,district_status,type_urban,type_suburban,type_town,type_rural,eligible,teachers_nodegree,teachers_badegree,teachers_msdegree,teachers_phddegree,r_3rd_std,m_3rd_std,r_4th_std,m_4th_std,r_5th_std,m_5th_std,r_6th_std,m_6th_std,r_7th_std,m_7th_std,r_8th_std,m_8th_std,s_8th_std,alg_std,bio_std,eng1_std,eng2_std,us_std,elem_math,elem_reading,elem,middle_math,middle_reading,middle_science,algebra,biology,eng1,avescores,treatpost,yearpost,yearpre,pre5,pre4,pre3,pre2,pre1,post1,post2,post3,pre_num,pre_tenure,pre_turnover,pre

In [101]:
def model_to_excel(data, y, x_list, file, start_row, start_col):
    wb = load_workbook(file)
    ws = wb.active

    col_n = start_col
    row_n = start_row

    # create formula
    i = 1
    x_str = ''
    for x in x_list:
        if i == 1:
            x_str = x_str + x
        if i > 1:
            x_str = x_str + ' + ' + x
        i += 1
    x_str = x_str + '- 1'
    formula = y + ' ~ ' + x_str

    # run
    result = smf.logit(formula=formula, data=data).fit()
    # to excel
    for x in x_list:
        p = result.pvalues[x]
        coef = result.params[x].round(2)
        if p >= .05:
            coef = str(coef)
        if p < .05 and p > .01:
            coef = str(coef) + '*'
        if p < .01 and p > .001:
            coef = str(coef) + '**'
        if p < .001:
            coef = str(coef) + '***'
        ws.cell(row=row_n, column=col_n).value = coef
        row_n = row_n + 1
        ws.cell(row=row_n, column=col_n).value = '(' + str(result.bse[x].round(2)) + ')'
        row_n = row_n + 1

    ws.cell(row=row_n, column=col_n).value = result.nobs
    row_n = row_n + 1
    ws.cell(row=row_n, column=col_n).value = result.llf.round(2)
    row_n = row_n + 1
    ws.cell(row=row_n, column=col_n).value = result.prsquared.round(2)
    wb.save(file)
    
    print(result)

    return result.summary()

In [102]:
x_list = ['pre_rural','pre_turnover', 'pre_tenure', 'pre_sturatio',
           'pre_hisp', 'pre_black', 'pre_frpl', 'pre_avescore']

# Schedules

In [87]:
tables.model_to_excel(data, y = 'reg25_0811', x_list = x_list, 
                      file = table_path + 'logit_schedules.xlsx', start_row = 4, start_col = 2)

Optimization terminated successfully.
         Current function value: 0.126961
         Iterations 8


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:             reg25_0811   No. Observations:                 5569
Model:                          Logit   Df Residuals:                     5562
Method:                           MLE   Df Model:                            6
Date:                Sat, 15 Feb 2020   Pseudo R-squ.:               -0.003750
Time:                        11:44:40   Log-Likelihood:                -707.05
converged:                       True   LL-Null:                       -704.41
                                        LLR p-value:                     1.000
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
pre_turnover     4.7553      1.068      4.452      0.000       2.662       6.849
pre_tenure       0.0305      0.039      0.783      0.434      -0.046       0.107
pre_sturatio     0.1953      0.029      6.697      0.000       0.138       0.252
pre_hisp        -1.5285      0.418     -3.657      0.000      -2.348      -0.709
pre_black        1.2164      1.220      0.997      0.319      -1.175       3.608
pre_frpl         0.7194      0.668      1.077      0.282      -0.590       2.029
pre_avescore     0.4358      0.149      2.931      0.003       0.144       0.727
================================================================================
"""

In [88]:
tables.model_to_excel(data, y = 'reg25_0811', x_list = x_list, 
                      file = table_path + 'logit_schedules.xlsx', start_row = 4, start_col = 2)
tables.model_to_excel(data, y = 'reg25_081', x_list = x_list, 
                      file = table_path + 'logit_schedules.xlsx', start_row = 4, start_col = 3)
tables.model_to_excel(data, y = 'reg25_0812', x_list = x_list, 
                      file = table_path + 'logit_schedules.xlsx', start_row = 4, start_col = 4)

Optimization terminated successfully.
         Current function value: 0.126961
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.674632
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.590809
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:             reg25_0812   No. Observations:                 5569
Model:                          Logit   Df Residuals:                     5562
Method:                           MLE   Df Model:                            6
Date:                Sat, 15 Feb 2020   Pseudo R-squ.:                0.009971
Time:                        11:44:40   Log-Likelihood:                -3290.2
converged:                       True   LL-Null:                       -3323.3
                                        LLR p-value:                 2.371e-12
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
pre_turnover     0.5987      0.378      1.585      0.113      -0.142       1.339
pre_tenure      -0.0067      0.016     -0.427      0.669      -0.038       0.024
pre_sturatio    -0.0859      0.011     -7.926      0.000      -0.107      -0.065
pre_hisp         0.5486      0.162      3.383      0.001       0.231       0.866
pre_black        0.3255      0.338      0.964      0.335      -0.336       0.987
pre_frpl        -0.2821      0.257     -1.097      0.273      -0.786       0.222
pre_avescore     0.0833      0.056      1.479      0.139      -0.027       0.194
================================================================================
"""

In [89]:
data[data.pre_urban == 1].reg25_0811.mean()

0.9487179487179487

In [90]:
data[data.pre_town == 1].reg25_0811.mean()

0.9816152330925805

# Certification and Class Sizes

In [91]:
tables.model_to_excel(data, y = 'reg21_003', x_list = x_list, 
                      file = table_path + 'logit_certification.xlsx', start_row = 4, start_col = 2)
tables.model_to_excel(data, y = 'reg25_112', x_list = x_list, 
                      file = table_path + 'logit_certification.xlsx', start_row = 4, start_col = 4)
tables.model_to_excel(data, y = 'reg25_111', x_list = x_list, 
                      file = table_path + 'logit_certification.xlsx', start_row = 4, start_col = 5)

Optimization terminated successfully.
         Current function value: 0.383539
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.675779
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.426189
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:              reg25_111   No. Observations:                 5569
Model:                          Logit   Df Residuals:                     5562
Method:                           MLE   Df Model:                            6
Date:                Sat, 15 Feb 2020   Pseudo R-squ.:               -0.005532
Time:                        11:44:41   Log-Likelihood:                -2373.4
converged:                       True   LL-Null:                       -2360.4
                                        LLR p-value:                     1.000
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
pre_turnover    -2.2282      0.506     -4.400      0.000      -3.221      -1.236
pre_tenure      -0.1454      0.020     -7.196      0.000      -0.185      -0.106
pre_sturatio     0.0331      0.014      2.407      0.016       0.006       0.060
pre_hisp         0.2290      0.207      1.106      0.269      -0.177       0.635
pre_black        0.5442      0.401      1.358      0.174      -0.241       1.330
pre_frpl        -1.3149      0.323     -4.076      0.000      -1.947      -0.683
pre_avescore    -0.2261      0.071     -3.177      0.001      -0.366      -0.087
================================================================================
"""

# Contracts

In [103]:
tables.model_to_excel(data, y = 'reg21_102', x_list = x_list, 
                      file = table_path + 'logit_contracts.xlsx', start_row = 4, start_col = 2)
tables.model_to_excel(data, y = 'reg21_401', x_list = x_list, 
                      file = table_path + 'logit_contracts.xlsx', start_row = 4, start_col = 3)
tables.model_to_excel(data, y = 'reg21_352', x_list = x_list, 
                     file = table_path + 'logit_contracts.xlsx', start_row = 4, start_col = 4)
tables.model_to_excel(data, y = 'reg21_354', x_list = x_list, 
                      file = table_path + 'logit_contracts.xlsx', start_row = 4, start_col = 5)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/kylieleblancKylie/domino/dofis/results/Who Needs Rules/logit_contracts.xlsx'

# Behavior

In [93]:
tables.model_to_excel(data, y = 'reg25_092', x_list = x_list, 
                      file = table_path + 'logit_behavior.xlsx', start_row = 4, start_col = 2)
tables.model_to_excel(data, y = 'reg37_0012', x_list = x_list, 
                      file = table_path + 'logit_behavior.xlsx', start_row = 4, start_col = 3)
tables.model_to_excel(data, y = 'reg25_036', x_list = x_list, 
                      file = table_path + 'logit_behavior.xlsx', start_row = 4, start_col = 4)

Optimization terminated successfully.
         Current function value: 0.553665
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.505303
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.494087
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:              reg25_036   No. Observations:                 5569
Model:                          Logit   Df Residuals:                     5562
Method:                           MLE   Df Model:                            6
Date:                Sat, 15 Feb 2020   Pseudo R-squ.:                 0.04763
Time:                        11:44:41   Log-Likelihood:                -2751.6
converged:                       True   LL-Null:                       -2889.2
                                        LLR p-value:                 1.670e-56
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
pre_turnover     1.2614      0.405      3.116      0.002       0.468       2.055
pre_tenure      -0.0181      0.017     -1.039      0.299      -0.052       0.016
pre_sturatio    -0.1696      0.012    -13.794      0.000      -0.194      -0.146
pre_hisp        -1.0574      0.180     -5.877      0.000      -1.410      -0.705
pre_black       -1.5812      0.408     -3.873      0.000      -2.381      -0.781
pre_frpl         1.8735      0.283      6.623      0.000       1.319       2.428
pre_avescore     0.3243      0.063      5.157      0.000       0.201       0.448
================================================================================
"""